In [1]:
import graphlab as gl
import pandas as pd
import numpy as np
from graphlab import SFrame, SGraph

In [2]:
transactions = pd.read_csv('tidValues.csv')

In [3]:
transactions = SFrame(transactions)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1550960610.log


This non-commercial license of GraphLab Create for academic use is assigned to krupal.jadhav@wsu.edu and will expire on November 06, 2019.


In [30]:
transactions

tid,asin
1,0827229534
1,0804215715
1,156101074X
1,0687023955
1,0687074231
1,082721619X
2,0738700797
2,0738700827
2,1567184960
2,1567182836


In [5]:
asins_df = transactions.select_columns(['asin']).to_dataframe()
unique_asins = asins_df.asin.unique()

In [6]:
#Write to Asins CSV
pd.DataFrame(unique_asins, columns=['asin']).to_csv('Asins.csv', index=False)

In [ ]:
model = gl.frequent_pattern_mining.create(transactions, 'asin', features=['tid'], min_length=4, max_patterns=500)

In [8]:
model.get_frequent_patterns().export_csv('frequent_patterns.csv', '\t')

In [9]:
model.save('fpm_model')

In [10]:
def getMainAsin(pred_table_df):
    flattened_prefix = [val for sublist in pred_table_df.prefix for val in sublist]
    unique_val_prefix = list(set(list(flattened_prefix)))
    unique_val_prefix = "".join(unique_val_prefix)
    return unique_val_prefix

In [129]:
def getPredictionsConfidences(pred_table_df):
    flattened_predictions = [val for sublist in pred_table_df.prediction for val in sublist]
    unique_val_predictions  = list(set(list(flattened_predictions)))
    unique_val_predictions = ",".join(unique_val_predictions)
    
    confidences = []
    predictions = []
    prediction_confidence_map = zip(pred_table_df.prediction, pred_table_df.confidence)
    for item in list((unique_val_predictions).split(',')):
        for i in range(len(prediction_confidence_map)):
            if (item in ','.join(prediction_confidence_map[i][0])) and (item not in predictions):
                predictions.append(item)
                confidences.append(round(prediction_confidence_map[i][1],2))
                
    return ','.join(predictions), ','.join(str(a) for a in confidences)

In [256]:
%%capture
asins = []
predictions = []
confidences = []

for i, j in zip(start, end):
    count = count+1
    print('recommend'+str(count))
    for asin_value in unique_asins[i:j]:
        pred_table = model.predict_topk(SFrame(pd.DataFrame([['null', asin_value]], columns = ['tid','asin'])), k = 5)
        pred_table_df = pred_table.to_dataframe()
        asin = getMainAsin(pred_table_df)
        if asin:
            pred, conf = getPredictionsConfidences(pred_table_df)
            asins.append(asin)
            predictions.append(pred)
            confidences.append(conf)
    data = zip(asins, predictions, confidences)
    df = pd.DataFrame(data, columns=['asin', 'predictions', 'confidence'])
    df.to_csv('sam_recommend'+str(count)+'.tsv', sep = '\t', index=False)
    asins = []
    predictions = []
    confidences = []

RuntimeError: Runtime Exception. Canceled by user

In [309]:
print(np.asarray(start))
print()
print(np.asarray(end))

[     0  10000  20000  30000  40000  50000  60000  70000  80000  90000
 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000
 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000
 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000]
()
[ 10000  20000  30000  40000  50000  60000  70000  80000  90000 100000
 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000
 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000
 310000 320000 330000 340000 350000 360000 370000 380000 390000 393562]


In [ ]:
%%capture
asins = []
predictions = []
confidences = []
count = 0

for asin_value in unique_asins[50000:60000]:
    pred_table = model.predict_topk(SFrame(pd.DataFrame([['null', asin_value]], columns = ['tid','asin'])), k = 5)
    pred_table_df = pred_table.to_dataframe()
    asin = getMainAsin(pred_table_df)
    if asin:
        pred, conf = getPredictionsConfidences(pred_table_df)
        asins.append(asin)
        predictions.append(pred)
        confidences.append(conf)

In [322]:
f = open('output4.txt', 'w+')

for a, p, c in zip(asins, predictions, confidences):
    a = "insert into recommendation (asin, prediction, confidence) " + "values ('" + a + "','" + p + "','" + c +"');"
    f.write(a)
    f.write('\n')
f.close()

In [ ]:
np.asarray(end)

In [260]:
data = zip(asins, predictions, confidences)
df = pd.DataFrame(data, columns=['asin', 'predictions', 'confidence'])
df

'0.45,0.0,0.39,0.0,0.0'

In [304]:
len(df)

12

In [265]:
data = zip(asins, predictions, confidences)
df = pd.DataFrame(data, columns=['asin', 'predictions', 'confidence'])
df.to_csv('recommend'+str(count)+'.tsv', sep = '\t', index=False)

In [152]:
len(unique_asins) - 390000

3561

In [138]:
len(unique_asins[300001:-1])

93559

In [227]:
index = range(10000, 400000, 10000)
index.append(393562)

In [252]:
start = []
end = []
#start = [x+1 for x in start]

In [253]:
for i,j in zip(range(0, 400000, 10000), index):
    start.append(i)
    end.append(j)

In [254]:
#start[0] = start[0]-1


array([     0,  10000,  20000,  30000,  40000,  50000,  60000,  70000,
        80000,  90000, 100000, 110000, 120000, 130000, 140000, 150000,
       160000, 170000, 180000, 190000, 200000, 210000, 220000, 230000,
       240000, 250000, 260000, 270000, 280000, 290000, 300000, 310000,
       320000, 330000, 340000, 350000, 360000, 370000, 380000, 390000])

In [255]:
np.asarray(end)

array([ 10000,  20000,  30000,  40000,  50000,  60000,  70000,  80000,
        90000, 100000, 110000, 120000, 130000, 140000, 150000, 160000,
       170000, 180000, 190000, 200000, 210000, 220000, 230000, 240000,
       250000, 260000, 270000, 280000, 290000, 300000, 310000, 320000,
       330000, 340000, 350000, 360000, 370000, 380000, 390000, 393562])

In [161]:
len(range(0, 400000, 10000))

40

In [242]:
count=0
for i, j in zip(start, end):
    count = count+1
    print('recommend'+str(count)+'.tsv')

recommend1.tsv
recommend2.tsv
recommend3.tsv
recommend4.tsv
recommend5.tsv
recommend6.tsv
recommend7.tsv
recommend8.tsv
recommend9.tsv
recommend10.tsv
recommend11.tsv
recommend12.tsv
recommend13.tsv
recommend14.tsv
recommend15.tsv
recommend16.tsv
recommend17.tsv
recommend18.tsv
recommend19.tsv
recommend20.tsv
recommend21.tsv
recommend22.tsv
recommend23.tsv
recommend24.tsv
recommend25.tsv
recommend26.tsv
recommend27.tsv
recommend28.tsv
recommend29.tsv
recommend30.tsv
recommend31.tsv
recommend32.tsv
recommend33.tsv
recommend34.tsv
recommend35.tsv
recommend36.tsv
recommend37.tsv
recommend38.tsv
recommend39.tsv
recommend40.tsv


In [240]:
count

40

In [234]:
sum(a)

393561

In [190]:
len(unique_asins)

393561

In [ ]:
len(unique_asins[])